# Preprocessing

In [ ]:
# import os
# from random import randint, uniform
# import re
# import numpy as np
# import wave
# import contextlib

## Data Preparation for Recording

In [1]:
MAX_SENTENCE_LEN = 20
SOURCE_PATH = 'yor_trans.txt'
DEST_TEMP_PATH = 'yor_split.txt'
DEST_TEMP_CLEAN_PATH = 'yor_clean_split.txt'
NUM_FOLDER_SPLIT = 50
FOLDER_PATH="split_text"
PARTIAL_NAME= "yor_split"

In [2]:
!mkdir {FOLDER_PATH}

mkdir: cannot create directory ‘split_text’: File exists


In [3]:
from yor_processor import split_file, split_file_into_folders

In [4]:
status = split_file(SOURCE_PATH, DEST_TEMP_PATH, MAX_SENTENCE_LEN)
# clean_file_status = split_file(SOURCE_PATH, DEST_TEMP_CLEAN_PATH, MAX_SENTENCE_LEN, end_of_file="\n")
if status == 'done':
    split_file_into_folders(FOLDER_PATH, DEST_TEMP_PATH, PARTIAL_NAME, NUM_FOLDER_SPLIT)

## Calculate Total Recording Lengths

In [5]:
DIR_OF_REC = "./recordings/"
FILE_FORMAT = ".wav"

In [6]:
from yor_processor import calculate_recording_len

total_len, good_files, corrupted_files = calculate_recording_len(DIR_OF_REC, FILE_FORMAT)
print("total len of recording is ", round(total_len, 2), "s",", ",total_len/(60*60),"h")

total len of recording is  11030.87 s ,  3.0641296354166636 h


In [7]:
print(good_files)
print(corrupted_files)

1079
30


# Organizing data

### Spliting records to train, val and test sets

In [8]:
!mkdir data
!mkdir data/records

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/records’: File exists


In [9]:
from yor_processor import extract_non_corrupted_files

wav_files, linker = extract_non_corrupted_files(DIR_OF_REC)

In [10]:
# copy wav files
to_copy = " ".join(wav_files)
!cp -t data/records/ {to_copy}

In [11]:
# add linker files
for i, link in enumerate(linker):
    line = link.split(";")[0].split("(")[1].split(")")[0].strip()
    wav = link.split(";")[1].strip().split("/")[-1]
    linker[i] = wav+":"+line

In [12]:
out_linker = open("./data/linker.txt","w")
out_linker.write("\n".join(linker))
out_linker.close()

In [13]:
!mkdir ./data/records/train
!mkdir ./data/records/test
!mkdir ./data/records/val
!mkdir ./data/records/extra

mkdir: cannot create directory ‘./data/records/train’: File exists
mkdir: cannot create directory ‘./data/records/test’: File exists
mkdir: cannot create directory ‘./data/records/val’: File exists
mkdir: cannot create directory ‘./data/records/extra’: File exists


In [14]:
from yor_processor import split_train_val_test

# reduce to 2hrs data since I have 3hrs of data
# split 1hr into train and val
to_copy_train, to_copy_valid, to_copy_test, to_copy_extra = split_train_val_test(wav_files, 
                                                                                 num_splits=3, # reduce to 2hrs data since I have 3hrs of data
                                                                                 val_split=0.2)

In [15]:
!mv -t data/records/train/ {to_copy_train}
!mv -t data/records/val/ {to_copy_valid}
!mv -t data/records/test/ {to_copy_test}
!mv -t data/records/extra/ {to_copy_extra}

### Make chars.txt file

In [16]:
from yor_processor import create_char_set

chars_dict, text_data  = create_char_set(linker, path="./yor_split.txt", exclude="[##\n\.,''-''̀''́''-']")

In [17]:
print(chars_dict)

{'ε': 0, ' ': 1, 'y': 2, 'à': 3, 'n': 4, 'r': 5, 'j': 6, 'ọ': 7, 'ì': 8, 'ù': 9, 'd': 10, 'i': 11, 'u': 12, 'm': 13, 'b': 14, 'e': 15, 'ẹ': 16, 'l': 17, 'o': 18, 'ò': 19, 't': 20, 'a': 21, 'g': 22, 's': 23, 'f': 24, 'è': 25, 'k': 26, 'á': 27, 'é': 28, 'ó': 29, 'w': 30, 'í': 31, 'p': 32, 'ṣ': 33, 'ú': 34, 'h': 35, '-': 36}


### Create data in format for training

In [18]:
from yor_processor import create_data_format

create_data_format(text_data, chars_dict)

files created for training
